In [1]:
import xarray as xr
import pandas as pd

In [ ]:
ds_mlev= xr.open_dataset("/path/to/data_mlev.nc")
ds_mlev

<xarray.Dataset> Size: 692kB
Dimensions:      (valid_time: 17288, model_level: 1, latitude: 2, longitude: 4)
Coordinates:
  * valid_time   (valid_time) datetime64[ns] 138kB 2019-01-01 ... 2024-11-30T...
  * model_level  (model_level) float64 8B 60.0
  * latitude     (latitude) float64 16B 45.75 45.0
  * longitude    (longitude) float64 32B 8.0 8.75 9.5 10.25
Data variables:
    co           (valid_time, model_level, latitude, longitude) float32 553kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-06T13:25 GRIB to CDM+CF via cfgrib-0.9.1...

In [3]:
cams_CO=ds_mlev.to_dataframe()
cams_CO.reset_index(drop=False, inplace=True)
cams_CO.drop(columns={'model_level'},inplace=True)
cams_CO.rename(columns={'co':'cams_CO'},inplace=True)
cams_CO

,valid_time,latitude,longitude,cams_CO
0,2019-01-01 00:00:00,45.75,8.00,2.587038e-07
1,2019-01-01 00:00:00,45.75,8.75,3.156148e-07
2,2019-01-01 00:00:00,45.75,9.50,3.001970e-07
3,2019-01-01 00:00:00,45.75,10.25,2.796525e-07
4,2019-01-01 00:00:00,45.00,8.00,4.167173e-07
...,...,...,...,...
138299,2024-11-30 21:00:00,45.75,10.25,2.904796e-07
138300,2024-11-30 21:00:00,45.00,8.00,3.041743e-07
138301,2024-11-30 21:00:00,45.00,8.75,2.882353e-07
138302,2024-11-30 21:00:00,45.00,9.50,2.691882e-07


In [4]:
# convert time zone
cams_CO['valid_time'] = cams_CO['valid_time'].dt.tz_localize('UTC')      
cams_CO['valid_time'] = cams_CO['valid_time'].dt.tz_convert('Europe/Rome')
cams_CO

,valid_time,latitude,longitude,cams_CO
0,2019-01-01 01:00:00+01:00,45.75,8.00,2.587038e-07
1,2019-01-01 01:00:00+01:00,45.75,8.75,3.156148e-07
2,2019-01-01 01:00:00+01:00,45.75,9.50,3.001970e-07
3,2019-01-01 01:00:00+01:00,45.75,10.25,2.796525e-07
4,2019-01-01 01:00:00+01:00,45.00,8.00,4.167173e-07
...,...,...,...,...
138299,2024-11-30 22:00:00+01:00,45.75,10.25,2.904796e-07
138300,2024-11-30 22:00:00+01:00,45.00,8.00,3.041743e-07
138301,2024-11-30 22:00:00+01:00,45.00,8.75,2.882353e-07
138302,2024-11-30 22:00:00+01:00,45.00,9.50,2.691882e-07


In [5]:
# Convert 'valid_time' to datetime and floor to daily frequency (keeps datetime64 type)
cams_CO['date'] = cams_CO['valid_time'].dt.floor('D')
cams_CO['hour'] = cams_CO['valid_time'].dt.hour

# Filter data from 21pm previous day to 15pm current day
filtered = cams_CO[(cams_CO['hour'] >= 21) | (cams_CO['hour'] <= 15)]

# Shift dates for hours >=21 to the next day
filtered['group_date'] = (
    filtered['date'] + pd.to_timedelta((filtered['hour'] >= 21).astype(int), unit='D')
).dt.date  # Now works because 'date' is datetime64

# Group by adjusted date and compute mean
cams_CO = filtered.groupby(['group_date', 'latitude','longitude']).agg({'cams_CO': 'mean'}).reset_index()
cams_CO.rename(columns={'group_date': 'date'}, inplace=True)

cams_CO

/tmp/ipykernel_20575/2437267390.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered['group_date'] = (


,date,latitude,longitude,cams_CO
0,2019-01-01,45.00,8.00,3.742165e-07
1,2019-01-01,45.00,8.75,5.875754e-07
2,2019-01-01,45.00,9.50,5.860836e-07
3,2019-01-01,45.00,10.25,5.013875e-07
4,2019-01-01,45.75,8.00,2.419757e-07
...,...,...,...,...
17291,2024-12-01,45.00,10.25,2.855481e-07
17292,2024-12-01,45.75,8.00,2.987221e-07
17293,2024-12-01,45.75,8.75,2.783389e-07
17294,2024-12-01,45.75,9.50,2.893257e-07


In [ ]:
ds_humidity=xr.open_dataset("/path/to/data_plev.nc")
ds_humidity

<xarray.Dataset> Size: 692kB
Dimensions:         (valid_time: 17288, pressure_level: 1, latitude: 2,
                     longitude: 4)
Coordinates:
  * valid_time      (valid_time) datetime64[ns] 138kB 2019-01-01 ... 2024-11-...
  * pressure_level  (pressure_level) float64 8B 1e+03
  * latitude        (latitude) float64 16B 45.75 45.0
  * longitude       (longitude) float64 32B 8.0 8.75 9.5 10.25
Data variables:
    r               (valid_time, pressure_level, latitude, longitude) float32 553kB ...
Attributes:
    GRIB_centre:             ecmf
    GRIB_centreDescription:  European Centre for Medium-Range Weather Forecasts
    GRIB_subCentre:          0
    Conventions:             CF-1.7
    institution:             European Centre for Medium-Range Weather Forecasts
    history:                 2025-05-06T14:40 GRIB to CDM+CF via cfgrib-0.9.1...

In [7]:
humidity=ds_humidity.to_dataframe()
humidity.reset_index(drop=False, inplace=True)
humidity.drop(columns={'pressure_level'},inplace=True)
humidity

,valid_time,latitude,longitude,r
0,2019-01-01 00:00:00,45.75,8.00,83.951736
1,2019-01-01 00:00:00,45.75,8.75,74.779861
2,2019-01-01 00:00:00,45.75,9.50,79.793045
3,2019-01-01 00:00:00,45.75,10.25,81.867752
4,2019-01-01 00:00:00,45.00,8.00,67.836502
...,...,...,...,...
138299,2024-11-30 21:00:00,45.75,10.25,85.082077
138300,2024-11-30 21:00:00,45.00,8.00,79.784714
138301,2024-11-30 21:00:00,45.00,8.75,82.322311
138302,2024-11-30 21:00:00,45.00,9.50,78.636765


In [8]:
# convert time zone
humidity['valid_time'] = humidity['valid_time'].dt.tz_localize('UTC')      
humidity['valid_time'] = humidity['valid_time'].dt.tz_convert('Europe/Rome')
humidity

,valid_time,latitude,longitude,r
0,2019-01-01 01:00:00+01:00,45.75,8.00,83.951736
1,2019-01-01 01:00:00+01:00,45.75,8.75,74.779861
2,2019-01-01 01:00:00+01:00,45.75,9.50,79.793045
3,2019-01-01 01:00:00+01:00,45.75,10.25,81.867752
4,2019-01-01 01:00:00+01:00,45.00,8.00,67.836502
...,...,...,...,...
138299,2024-11-30 22:00:00+01:00,45.75,10.25,85.082077
138300,2024-11-30 22:00:00+01:00,45.00,8.00,79.784714
138301,2024-11-30 22:00:00+01:00,45.00,8.75,82.322311
138302,2024-11-30 22:00:00+01:00,45.00,9.50,78.636765


In [9]:
humidity['valid_time'] = pd.to_datetime(humidity['valid_time'])
humidity = humidity.set_index('valid_time')

# 1. Current Day Value (21:00 previous day to 15:00 current day)
# Create mask for current window
current_mask = (humidity.index.hour >= 21) | (humidity.index.hour < 15)
current_data = humidity[current_mask].copy()

# Adjust grouping dates for current window:
current_data['group_date'] = current_data.index.where(
    current_data.index.hour < 15,  # Keep times <15:00 as current date
    current_data.index + pd.Timedelta(days=1)  # Shift >=21:00 to next day
).floor('D')

# Group by adjusted date and location
current_grouped = current_data.groupby([
    'group_date',
    'latitude',
    'longitude'
]).mean().reset_index()

# 2. Previous Value (15:00-21:00 of previous day)
# Create mask for previous window
pre_mask = (humidity.index.hour >= 15) & (humidity.index.hour < 21)
pre_data = humidity[pre_mask].copy()

# Group by original date and location
pre_grouped = pre_data.groupby([
    pd.Grouper(freq='D'),
    'latitude',
    'longitude'
]).mean().reset_index()

# Shift previous values forward by 1 day to align with current
pre_grouped['group_date'] = pre_grouped['valid_time'] + pd.Timedelta(days=1)
pre_grouped = pre_grouped.drop(columns=['valid_time'])

# 3. Combine and Clean
# Merge current and previous values
combined = pd.merge(
    current_grouped,
    pre_grouped,
    on=['group_date', 'latitude', 'longitude'],
    suffixes=('', '_pre')
)

# Trim edge dates (first and last day may have incomplete data)
combined = combined[
    (combined['group_date'] > combined['group_date'].min()) &
    (combined['group_date'] < combined['group_date'].max())
]

# Final cleanup
combined = combined.rename(columns={'group_date': 'date'})

In [10]:
combined

,date,latitude,longitude,r,r_pre
8,2019-01-03 00:00:00+01:00,45.00,8.00,44.968307,40.587234
9,2019-01-03 00:00:00+01:00,45.00,8.75,40.874313,31.559402
10,2019-01-03 00:00:00+01:00,45.00,9.50,39.424118,29.675613
11,2019-01-03 00:00:00+01:00,45.00,10.25,34.543991,30.959061
12,2019-01-03 00:00:00+01:00,45.75,8.00,69.832161,57.127274
...,...,...,...,...,...
17179,2024-11-30 00:00:00+01:00,45.00,10.25,82.926308,73.609940
17180,2024-11-30 00:00:00+01:00,45.75,8.00,73.316566,67.825638
17181,2024-11-30 00:00:00+01:00,45.75,8.75,79.097496,69.898514
17182,2024-11-30 00:00:00+01:00,45.75,9.50,89.443153,73.291092


In [11]:
combined['date']=combined['date'].dt.date


In [12]:
cams=cams_CO.merge(combined,on=['date','latitude','longitude'],how='left')

In [13]:
cams

,date,latitude,longitude,cams_CO,r,r_pre
0,2019-01-01,45.00,8.00,3.742165e-07,NaN,NaN
1,2019-01-01,45.00,8.75,5.875754e-07,NaN,NaN
2,2019-01-01,45.00,9.50,5.860836e-07,NaN,NaN
3,2019-01-01,45.00,10.25,5.013875e-07,NaN,NaN
4,2019-01-01,45.75,8.00,2.419757e-07,NaN,NaN
...,...,...,...,...,...,...
17291,2024-12-01,45.00,10.25,2.855481e-07,NaN,NaN
17292,2024-12-01,45.75,8.00,2.987221e-07,NaN,NaN
17293,2024-12-01,45.75,8.75,2.783389e-07,NaN,NaN
17294,2024-12-01,45.75,9.50,2.893257e-07,NaN,NaN


In [ ]:
cams.to_feather("/path/to/CAMS_21_15.feather")